# RAG desde PDF - ESPAÑOL

Este RAG recibe tres pdfs sobre animales y responde a unas preguntas

## 1. Código

In [67]:
# Introducción documento "externo"
import PyPDF2
pdf_names=[ "Perros", "Cabras", "Gatos"]

pdf_paths=[]
for name in pdf_names:
    pdf_paths.append('pdfs/'+name+'.pdf')
text=""
for pdf in pdf_paths:
    with open(pdf, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text = text + page.extract_text() 

In [68]:
# Split del texto recibido
import langchain
from langchain.text_splitter  import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",  
    chunk_size=400,  
    # chunk_overlap=200 
)

splits = text_splitter.split_text(text)
print(len(splits))

12


In [69]:
# Vectorizar
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

from langchain_community.vectorstores import Chroma

vector_store = Chroma.from_texts(
    texts=splits,
    collection_name="ds_career",
    embedding=embeddings,
    persist_directory="./chroma_ds_career",
)

retriever = vector_store.as_retriever()

In [70]:
# Ollama 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Responde a la pregunta concisamente y en castellano en base a este contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)


## 2. Preguntas

In [71]:
chain.invoke("¿Cuántas horas duerme un gato?")

'Un gato puede dormir hasta 16 horas al día.'

In [72]:
chain.invoke("¿Puede un gato girar las orejas 180 grados?")

'No, no hay información disponible en el texto proporcionado sobre si los gatos pueden girar sus orejas 180 grados.'

In [73]:
chain.invoke("¿Qué es un gato?")

'Un gato es un animal mascota conocido por sus características únicas, como su nariz única e importante para la orientación, sus bigotes para medir espacios, su capacidad para ver en la oscuridad y sus propiedades terapéuticas.'

In [74]:
chain.invoke("¿Tienen las cabras pupilas circulares?")

'No, las cabras tienen pupilas rectangulares.'

In [75]:
chain.invoke("¿Los perros pueden entender 400 palabras y gestos?")

'Sí, según el texto proporcionado, los perros pueden reconocer hasta 250 palabras y gestos. Por lo tanto, es probable que puedan entender un número similar o incluso menos de palabras y gestos.'

In [76]:
chain.invoke("¿Los perros tienen cuatro estómagos?")

'No, no hay información en el contexto proporcionado sobre que los perros tengan cuatro estómagos.'